Purpose: To access and parse a page inorder to make a dataframe out of a table!

Install and import "Requests" library for page access.

In [30]:
!conda install -c anaconda requests -y

Solving environment: done

# All requested packages already installed.



In [31]:
import requests as requests

In [32]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Download the Wikipedia Page

In [33]:
wikipedia_page = requests.get(wikipedia_link)
page = wikipedia_page.text
#print(page)

Install and Import "BeautifulSoup" for parsing of the page

In [34]:
!conda install -c anaconda BeautifulSoup4 -y

Solving environment: done

# All requested packages already installed.



In [35]:
from bs4 import BeautifulSoup

Install supporting html parsing libraries

In [36]:
!conda install -c anaconda lxml -y

Solving environment: done

# All requested packages already installed.



In [37]:
#!conda install -c anaconda html5 -y

Start parcing the page

In [38]:
soup = BeautifulSoup(page, 'lxml')

Segragate the table part of the page

In [39]:
#If we had several "table" tags/sections on one page, then we would use the next line
#which would use the "table" section which has class of XYZ.
#table_section = soup.find('table', class_='wikitable sortable')
#Since we do not have multiple "table" tags on one page, the next line
#would be the easier way of grabbing the first section (using "tbody" under "table" section)
table_section = soup.tbody
#print(soup.tbody.prettify())

Import Pandas

In [40]:
import pandas as pd

Parse the page-table and add each line to the new dataframe

In [41]:
res = []
for tr in table_section.find_all('tr'):
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
       res.append(row)

df = pd.DataFrame(res, columns=["Postcode","Borough","Neighborhood"])
df.head(15)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


First, lets drop rows that are "Not assigned" in the "Borough" column

In [42]:
#First, Drop rows that are "Not assigned" in "Borough" column 
df = df[df.Borough != 'Not assigned']
df.head(15)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Replace "Not Assigned" Neighborhood data with Borough values

In [43]:
#Purpose: Replace 'Not Assigned' Neighborhood data with Borough values
#
#This line of code works fine but needs Numpy libraray and should be a little faster
#>>> df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])

#This line of code works fine but in different way and uses Pandas
#df.Neighborhood[df.Neighborhood == 'Not assigned'] = df.Borough

#I decided to used this type of code because it seems simpler
df.Neighborhood.replace('Not assigned',df.Borough,inplace=True)
df.head(15)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Combine "Neighborhood" data/rows that have the same Postcode/Borough

In [45]:
#Combine "Neighborhood" data that have the same Borough
#Note: We need to use the 'apply' method to do this type of grouping
#This Line of code will work fine
#df_group = df.groupby(['Postcode', 'Borough']).apply(lambda group: ','.join(group['Neighborhood']))
#I found this line of code which seems to be a little simpler
df_grouped = df.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
df_grouped.head(15)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


Finally, show the shape of the final grouped dataframe !

In [46]:
df_grouped.shape

(103, 3)